Data  Validation Notebook

In [1]:
import os


In [2]:
%pwd
os.chdir("../")
%pwd

'/home/dephinate/ASU/TextSummarization'

1) Make an Entity for configurations

In [3]:
from dataclasses import dataclass
from pathlib import Path

In [4]:
@dataclass
class DataValidationConfig:
    root_dir:Path
    status_file:str
    all_required_files:list
    local_data_folder: Path

Update Configuration manager to fetch configurations from config.yaml and return a configuration object

In [5]:
from TextSummarizer.logging import logger
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories


In [6]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            param_filepath = PARAMS_FILE_PATH
            ) -> None:
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(param_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation

        create_directories([config.root_dir])
        
        data_validation_config = DataValidationConfig(
            root_dir= config.root_dir ,
            status_file= config.status_file,
            all_required_files= config.all_required_files,
            local_data_folder= config.local_data_folder
        ) 
        return data_validation_config
    


Create Data Validation Component to perform actual data Validation

In [22]:
class DataValidation:
    def __init__(
            self,
            config:DataValidationConfig
                 ) -> None:
        self.config = config

    def validate_all_files_exist(self):

        try:  

            validation_status = None
            files_in_local_data_folder = os.listdir(self.config.local_data_folder)
        
            for file in self.config.all_required_files:
                if file not in files_in_local_data_folder:
                    validation_status = False
                    with open(self.config.status_file, 'w') as f:
                        f.write(f"Validation Status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.status_file, 'w') as f:
                        f.write(f"Validation Status: {validation_status}")
            
            return validation_status
        
        except Exception as e:
            raise e

Update Pipeline

In [23]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config= data_validation_config)
    data_validation_status = data_validation.validate_all_files_exist()
except Exception as e:
    raise e

[2024-01-11 17:53:08,781,INFO,common,yaml file config/config.yaml loaded successfully]
[2024-01-11 17:53:08,800,INFO,common,yaml file params.yaml loaded successfully]
[2024-01-11 17:53:08,801,INFO,common,created directory at: artifacts]
[2024-01-11 17:53:08,801,INFO,common,created directory at: artifacts/data_validation]
